In [60]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import AdamWeightDecay


In [61]:

csv_path = "/content/drive/MyDrive/Restaurant_Reviews.csv"

df = pd.read_csv(csv_path, sep="\t", header=0)
df = df.rename(columns={"Review": "text", "Liked": "label"})
df["label"] = df["label"].astype(int)

print(df.shape)
df.head()


(1000, 2)


,text,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [62]:
X_train, X_val, y_train, y_val = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

print(len(X_train), len(X_val))


800 200


In [63]:
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

MAX_LEN = 128

train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=MAX_LEN
)

val_encodings = tokenizer(
    X_val,
    truncation=True,
    padding=True,
    max_length=MAX_LEN
)


In [68]:
train_dataset = tf.data.Dataset.from_tensor_slices({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": y_train
}).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": y_val
}).batch(16)


In [73]:
model = TFDistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    from_pt=True
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

In [74]:
optimizer = AdamWeightDecay(
    learning_rate=2e-5,
    weight_decay_rate=0.01
)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
    run_eagerly=True   # 🔑 KEY FIX
)



In [75]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)


Epoch 1/3
50/50 [==============================] - 23s 409ms/step - loss: 0.4981 - accuracy: 0.8000 - val_loss: 0.2674 - val_accuracy: 0.9100
Epoch 2/3
50/50 [==============================] - 20s 394ms/step - loss: 0.1385 - accuracy: 0.9575 - val_loss: 0.2190 - val_accuracy: 0.9250
Epoch 3/3
50/50 [==============================] - 21s 411ms/step - loss: 0.0509 - accuracy: 0.9887 - val_loss: 0.2662 - val_accuracy: 0.9300


In [76]:
import tensorflow as tf
import numpy as np

label_map = {
    0: "Negative",
    1: "Positive"
}

def predict_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensors="tf",
        truncation=True,
        padding=True,
        max_length=128
    )

    outputs = model(**inputs)
    probs = tf.nn.softmax(outputs.logits, axis=1)

    pred_label = int(tf.argmax(probs, axis=1)[0])
    confidence = float(tf.reduce_max(probs) * 100)

    return {
        "text": text,
        "sentiment": label_map[pred_label],
        "confidence": round(confidence, 2)
    }


In [77]:
tests = [
    "The food was amazing and the service was outstanding!",
    "This was the worst restaurant experience of my life.",
    "The place looks nice but the food was average.",
    "I waited for an hour and the food was cold.",
    "Absolutely loved it, will definitely come back!"
]

for t in tests:
    print(predict_sentiment(t))


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


{'text': 'The food was amazing and the service was outstanding!', 'sentiment': 'Positive', 'confidence': 99.31}
{'text': 'This was the worst restaurant experience of my life.', 'sentiment': 'Negative', 'confidence': 99.14}
{'text': 'The place looks nice but the food was average.', 'sentiment': 'Negative', 'confidence': 71.12}
{'text': 'I waited for an hour and the food was cold.', 'sentiment': 'Negative', 'confidence': 99.17}
{'text': 'Absolutely loved it, will definitely come back!', 'sentiment': 'Positive', 'confidence': 99.27}


In [78]:
tricky_tests = [
    "Yeah, great service... if you enjoy being ignored.",
    "Not bad, but not good either.",
    "I could care less about the food, the ambiance was nice.",
    "So amazing that I will never step foot here again."
]

for t in tricky_tests:
    print(predict_sentiment(t))


{'text': 'Yeah, great service... if you enjoy being ignored.', 'sentiment': 'Positive', 'confidence': 91.24}
{'text': 'Not bad, but not good either.', 'sentiment': 'Negative', 'confidence': 98.79}
{'text': 'I could care less about the food, the ambiance was nice.', 'sentiment': 'Positive', 'confidence': 99.24}
{'text': 'So amazing that I will never step foot here again.', 'sentiment': 'Positive', 'confidence': 99.21}


In [79]:
save_path = "/content/drive/MyDrive/lumina_sentiment_transformer"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Model and tokenizer saved at:", save_path)


✅ Model and tokenizer saved at: /content/drive/MyDrive/lumina_sentiment_transformer


In [80]:
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
import tensorflow as tf

reloaded_model = TFDistilBertForSequenceClassification.from_pretrained(
    save_path
)

reloaded_tokenizer = DistilBertTokenizerFast.from_pretrained(
    save_path
)

def predict_with_loaded_model(text):
    inputs = reloaded_tokenizer(
        text,
        return_tensors="tf",
        truncation=True,
        padding=True,
        max_length=128
    )
    outputs = reloaded_model(**inputs)
    probs = tf.nn.softmax(outputs.logits, axis=1)
    label = int(tf.argmax(probs, axis=1)[0])
    confidence = float(tf.reduce_max(probs) * 100)
    return label, round(confidence, 2)

print(predict_with_loaded_model("The food was amazing!"))
print(predict_with_loaded_model("Worst service ever."))


Some layers from the model checkpoint at /content/drive/MyDrive/lumina_sentiment_transformer were not used when initializing TFDistilBertForSequenceClassification: ['dropout_379']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/lumina_sentiment_transformer and are newly initialized: ['dropout_399']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(1, 99.3)
(0, 99.17)
